# Coding Exercise #6 *Modelocking, time-bandwidth-products, and pulse broadening due to material dispersion.*

### 6.1 Modelocking

**a)** Write a function `Et_LM(t, d, m, A, phi)` that returns the field, sampled on time axis `t`, of the `m`th longitudinal mode (LM) of a cavity of length `d`. `A` and `phi` are the amplitude and phase offset of the LM, respectively. You may assume that the LM is a cw wave.

**b)** Write a function `Et_LM_sum(t, d, m1, m2, phi=None)` which returns the coherent superposition (sum) of all LMs in the `m` range `m1` to `m2` for a cavity of length `d`, where `m` is the LM mode number. If the optional argument `phi` is omitted, then the phase offset of each LM should bet set to a random number in the range $0-2\pi$ radians, else the phase offset should be set to `phi`. You may assume that each LM has an amplitude of 1.

**c)** Consider a Nd:YAG laser with a cavity length of $1\,$m. LMs in the range $\pm 1\,$nm of its gain centre ($1064\,$nm) are lasing. Plot the laser ouptput intensity $I(t)\propto|E(t)|^2$ in the time-interval $0-30\,$ns, using ~10000 points to ensure adequate sampling for (i) random phase offsets between all LMs (ii) constant phase-offset between all LMs, i.e. *mode-locked*. You may assume all LMs have an amplitude of 1 and the output intensity can be displayed in arbitrary units.

Record your results.

**d)** Show that the pulse spacing in the modelocked train is given by $T_{rep}=2d/c$.

Hint: you can do this by inspecting the plot carefully, or write some code to find the peak positions, eg. using `find_peaks` from `scipy.signal`.

**e)** Recalculate and plot the mode-locked output but now zoom in on a single peak in the pulse train and use a new time axis to ensure the pulse has ~200 samples.

**f)** Show the width of the peaks in the mode-locked train is ~$\frac{2\pi}{N\Delta\omega_m}$, where $N$ is the number of LMs involved and $\Delta\omega_m$ is the LM spacing. 

Hint: You could reuse your `get_width` function from Exercise # 1.

**NOTE**: There is an error in the lecture notes for this formula! The $2\pi$ is missing in the numerator - sorry!

### 6.2 Time-Bandwidth Products

**a)** Write functions `Et_gauss(t, E0, dt_fwhm, lambda0)` and `Et_sech(t, E0, dt_fwhm, lambda0)` that return the electric fields of a transform-limited gaussian and sech pulse, respectively, where `t` is the time axis which the field will be sampled on, `E0` is the peak field, `dt_fwhm` is the fwhm duration for the intensity pulse $I(t)$, and `lambda0` is the centre wavelength.

Plot the field and the intensity as a function of time for $10\,$fs pulses with a centre wavelength of  $800\,$nm for both the gaussian and sech cases.

**b)** By taking the fourier transform of a $10\,$fs FWHM tranform-limited laser pulse, show that its time-bandwidth product $\Delta \nu \Delta t\approx 0.44$.

Repeat for a sech pulse to show its time-bandwidth product $\Delta \nu \Delta t\approx 0.32$.

**Hints**

* You can use `scipy.fft` for this. Feel free to use the function below which uses `scipy.fft` or write your own!
* use a time axis from -100 to 100 fs with 1024 points.
* If you don't get an accurate answer, it might be because you don't have enough points in your curves to measure the FWHMs accurately, e.g. using `get_width`. If this is the case, you can use `np.interp` to make an interpolating function to your curves which you can then sample more finely.


In [1]:
from scipy.fft import fft, fftfreq, fftshift
def get_intensity_spectrum(t, Et):
    """ 
    Uses scipy.fft to calculate the intensity spectrum of a laser pulse E(t). 
    `t` is time axis, `Et` is (complex) laser electric field sampled on t
    returns tuple (`omega`, `I`), where `omega` is angular frequency and `I` is spectral intensity.
    
    Tip: use 1024 or 2048 time points
    """
    assert len(t) == len(Et)
 
    t = np.array(t)
    Et = np.array(Et)
    
    N = len(t) # num points
    dt = t[1]-t[0] # time step
    f = fftfreq(N, dt) # frequency axis
    f = fftshift(f) # shift zero-frequency component to center of spectrum
    omega = 2 * np.pi * f # angular frequency axis

    Ef = fft(Et) # the fft
    Ef = fftshift(Ef) # shift zero-frequency component to center of spectrum
    I = np.abs(Ef)**2
    I /= max(I)
   
    return omega, I

### 6.3 Pulse broadening due to material dispersion

**a)** Write a function `n_fused_silica(lam)` that returns the refractive index of "fused silica" (a type of glass commonly used in optics) as a function of the wavelength `lam` in metres.

The formula can be found at 
[here](https://refractiveindex.info/?shelf=glass&book=fused_silica&page=Malitson). If you scroll down to the bottom of the page you will see `Expressions for n`, note that the wavelength is in microns.

**b)** Check your function against the calculator on the webpage.

**c)** Create a `numpy` array `omega` that covers the wavelength range $3000-200\,$nm in 1000 steps. Then create a `numpy` array `n` that is the refractive index of fused silica sampled on `omega`. Plot `n` vs `omega`.

**d)** Use the function `dydx` below to generate `numpy` arrays `dndomega` and `d2ndomega2` that are the first and second derivative of the refractive index, respectively, versus `omega`. Plot `dndomega` vs omega and `d2ndomega2` vs `omega` on separate plots.

In [2]:
def dydx(x, y):
    """ returns derivative with same shape as y, as opposed to np.diff which returns a shorter array """
    assert len(x) == len(y)
    x = np.array(x)
    y = np.array(y)
    dx = x[1] - x[0]
    return np.gradient(y, dx)

**e)** Construct a `numpy` array `vg` for the group velocity of light in fused silica, sampled on `omega` using the formula from the lecture notes:

### $v_g(\omega) =\frac{c}{n(\omega)+\omega \frac{dn(\omega)}{d\omega}}$.

Plot `vg` versus `omega` and record your observations.

**f)** Now create a `numpy` array `GVD` which is the group velocity dispersion sampled on `omega`. The GVD is defined as:

### $GVD = \frac{dv_g^{-1}}{d\omega}$.

Plot `GVD` in the commonly-used units of fs$^2$/mm vs `omega`.

**g)** A transform-limited gaussian pulse of duration `dt_in` propagates through a distance `L` of material with a GVD of `GVD`. Write a function `dt_out(dt_in, L, GVD)` that returns the broadened pulse duration. The relevant formulae from the lecture notes are:

### $\Delta t_{spread} = L*GVD*\Delta\omega$
### $\Delta t_{out} = \sqrt{\Delta t_{in}^2 + \Delta t_{spread}^2}$

A $5\,$fs transform-limited gaussian pulse propagating $1\,$mm through a material with a GVD of $35\,$fs$^2$/mm should be broadened to a duration of $20.0\,$fs. Use this to check your function.

## Extension

**h)** Because the GVD is frequency dependent, the amount of broadening that a laser pulse suffers depends on its centre frequency (centre wavelength).

Calculate and plot the broadening factor $\Delta t_{out}/\Delta t_{in}$ versus $\lambda_0$ in the range $200-3000\,$nm for an initially $5\,$fs transform-limited gaussian laser pulse propagating through $1\,$mm of fused silica, where the $\lambda_0$ is the centre wavelength of the pulse.

Hint: To allow you to calculate the GVD at an arbitrary frequency, use the function `interpolate.interp1d` from `scipy` to make an interpolating function to your previous `numpy` array `GVD`.